# Imports

In [1]:
from functions import *

# Setup Spark

In [2]:
from datetime import datetime

file = "data/US_Accidents_Dec19.csv"
model_indexing_dir = "log_index.md"
timeSignature = str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
models_dir = "models"
model_Note = 'Analysis tool'
images_dir = "analysis"
enabled = False
logger = logging("analysis", "analysis/logs.md", "analysis", str(timeSignature), enabled=enabled)

df,sc, spark = setup_spark(file=file)

In [3]:
sc

<SparkContext master=local[*] appName=Spark Project>

In [4]:
tmp = sc._conf.getAll()
logger.write2file("New Spark session", str(tmp))
print("Config:",tmp)

Config: [('spark.driver.memory', '4g'), ('spark.executor.memory', '4g'), ('spark.app.id', 'local-1588768733966'), ('spark.ui.enabled', 'true'), ('spark.executor.id', 'driver'), ('spark.app.name', 'Spark Project'), ('spark.ui.killEnabled', 'false'), ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'), ('spark.driver.host', '97901d576ca5'), ('spark.driver.port', '40147'), ('spark.rdd.compress', 'True'), ('spark.serializer.objectStreamReset', '100'), ('spark.executor.instances', '1'), ('spark.master', 'local[*]'), ('spark.executor.cores', '1'), ('spark.submit.deployMode', 'client'), ('spark.kryoserializer.buffer.max', '15'), ('spark.ui.showConsoleProgress', 'true'), ('spark.driver.cores', '1')]


# Define variables

In [5]:
colLabel = ["Severity"]

colRem = ['ID', 
          'Source',
          'End_Time',
          'End_Lat',
          'End_Lng',
          'Description',
        ]

df, colCat, colNum = setup_variables(df, sc, colLabel, colRem)    

logger.write2file("Number of rows", str(df.count()))
logger.write2file("Categorical groups","Defined Label:\n" + str(colLabel) + "\nDefined Categories:\n" + str(colCat) + "\nDefined Numerical:\n" +str(colNum))

Label: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight'] 
Categories: ['Street', 'Side', 'City', 'County', 'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code', 'Wind_Direction', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop', 'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight']
Numerical: ['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']


# 1. Analysis

In [6]:
print(f"Categories:\n \
Labels: {len(colLabel)}\n \
Classes: {len(colCat)}\n \
Removed: {len(colRem)}\n \
Numerical: {len(colNum)}") 

info = f"Rows: {df.count()}\nColumns {len(df.columns)}"
print(info)
df.printSchema()
df.take(1)

logger.write2file("Data analysis",info +"\n"+ str(df._jdf.schema().treeString()) + str(df.take(1)))

Categories:
 Labels: 1
 Classes: 28
 Removed: 6
 Numerical: 12
Rows: 2974335
Columns 43
root
 |-- TMC: double (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Number: double (nullable = true)
 |-- Street: string (nullable = true)
 |-- Side: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- Visibility(mi): double (nullable = true)

## Statistical summary

In [7]:
df.describe(colNum).show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+--------------------+
|summary|               TMC|         Start_Lat|         Start_Lng|      Distance(mi)|            Number|   Temperature(F)|    Wind_Chill(F)|       Humidity(%)|      Pressure(in)|   Visibility(mi)|  Wind_Speed(mph)|   Precipitation(in)|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+-----------------+-----------------+--------------------+
|  count|           2246264|           2974335|           2974335|           2974335|           1056730|          2918272|          1121712|           2915162|           2926193|          2908644|          2533495|              975977|
|   mean|207.83163198982845|36.493605002031636| -95.4262

## Get missing data

In [8]:
printMissingValues(df, logger)

Missing values[Row(TMC=728071, Severity=0, Start_Time=0, Start_Lat=0, Start_Lng=0, Distance(mi)=0, Number=1917605, Street=0, Side=0, City=83, County=0, State=0, Zipcode=880, Country=0, Timezone=3163, Airport_Code=5691, Weather_Timestamp=36705, Temperature(F)=56063, Wind_Chill(F)=1852623, Humidity(%)=59173, Pressure(in)=48142, Visibility(mi)=65691, Wind_Direction=45101, Wind_Speed(mph)=440840, Precipitation(in)=1998358, Weather_Condition=65932, Amenity=0, Bump=0, Crossing=0, Give_Way=0, Junction=0, No_Exit=0, Railway=0, Roundabout=0, Station=0, Stop=0, Traffic_Calming=0, Traffic_Signal=0, Turning_Loop=0, Sunrise_Sunset=93, Civil_Twilight=93, Nautical_Twilight=93, Astronomical_Twilight=93)]


## Get categorical values

In [9]:
df.persist()
tmp = [df.select(countDistinct(c).alias(c)).collect()[0] for c in [*colCat]] 
df.unpersist()
print("Unique column values:", tmp)

logger.write2file("Unique column values", str(tmp))

Unique column values: [Row(Street=160715), Row(Side=3), Row(City=11685), Row(County=1713), Row(State=49), Row(Zipcode=377152), Row(Country=1), Row(Timezone=4), Row(Airport_Code=1995), Row(Wind_Direction=24), Row(Weather_Condition=120), Row(Amenity=2), Row(Bump=2), Row(Crossing=2), Row(Give_Way=2), Row(Junction=2), Row(No_Exit=2), Row(Railway=2), Row(Roundabout=2), Row(Station=2), Row(Stop=2), Row(Traffic_Calming=2), Row(Traffic_Signal=2), Row(Turning_Loop=1), Row(Sunrise_Sunset=2), Row(Civil_Twilight=2), Row(Nautical_Twilight=2), Row(Astronomical_Twilight=2)]


## Correlation

In [10]:
print([*colNum, *colLabel])

['TMC', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Number', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)', 'Severity']


In [11]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat import Correlation

assembler = VectorAssembler(
    inputCols=colNum,
    outputCol="features",
    handleInvalid = "skip")

df_attributes = assembler.transform(df)
df_attributes.select("features").show(1,False)



r1 = Correlation.corr(df_attributes, "features").head()

print("correlation matrix:\n" + str(r1[0]))

logger.write2file("Correlation matrix", str(r1[0]))

+---------------------------------------------------------------------------+
|features                                                                   |
+---------------------------------------------------------------------------+
|[201.0,39.972038,-82.913521,0.01,3280.0,37.4,33.8,100.0,29.62,3.0,4.6,0.02]|
+---------------------------------------------------------------------------+
only showing top 1 row

correlation matrix:
DenseMatrix([[ 1.00000000e+00, -3.29968079e-02, -4.23709318e-02,
               6.48521350e-02,  4.88102771e-02,  2.89427459e-02,
               3.07391705e-02, -6.70493470e-03,  4.59513996e-03,
               2.21819466e-02, -1.61669692e-03, -1.80310455e-03],
             [-3.29968079e-02,  1.00000000e+00, -5.22492702e-03,
               5.01751676e-02, -3.17558810e-02, -4.68351247e-01,
              -4.69159899e-01,  9.79378608e-02, -1.18202158e-01,
              -1.54998940e-01,  9.15359600e-02,  5.03876348e-03],
             [-4.23709318e-02, -5.22492702e-

In [12]:
if enabled == True:
    sns.set(style="white")
    f, ax = plt.subplots(figsize=(11, 9))
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    b = sns.heatmap(r1[0].toArray().tolist(), annot=True, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5},xticklabels=colNum,yticklabels=colNum,ax=ax )
    ax.set_title("Correlation between numerical features")

    logger.saveImage(b,"feature_corr_matrix")

## Severity

In [13]:
severity_freq = df.groupBy('Severity').count().orderBy('count',ascending=False).collect()
severity_freq

[Row(Severity=2, count=1993410),
 Row(Severity=3, count=887620),
 Row(Severity=4, count=92337),
 Row(Severity=1, count=968)]

In [14]:
if enabled == True:
    rdd = sc.parallelize(severity_freq)
    pd_severity = rdd.toDF().toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))
    b = sns.barplot(pd_severity['Severity'],pd_severity['count'], color='blue')
    b.axes.set_title("Severity distribution",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("Severity",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"severity_dist")

In [15]:
if enabled == True:
    df_sev = df.groupby('Severity').count().toPandas()
    fig = df_sev.plot.pie(y='count', labels=df_sev['Severity'], figsize=(10, 10), autopct='%1.0f%%',title="Pie plot - Severity distribution",fontsize=15)
    fig.figure.savefig(images_dir +"/pie_severity_dist")

## Causes

### Frequency of severity and state

In [16]:
state_freq = df.groupBy('State').count().orderBy('count',ascending=False).collect()
state_freq

[Row(State='CA', count=663204),
 Row(State='TX', count=298062),
 Row(State='FL', count=223746),
 Row(State='SC', count=146689),
 Row(State='NC', count=142460),
 Row(State='NY', count=137799),
 Row(State='PA', count=90395),
 Row(State='MI', count=88694),
 Row(State='IL', count=86390),
 Row(State='GA', count=83620),
 Row(State='VA', count=79957),
 Row(State='OR', count=70840),
 Row(State='MN', count=62727),
 Row(State='AZ', count=62330),
 Row(State='WA', count=61367),
 Row(State='TN', count=58289),
 Row(State='OH', count=55863),
 Row(State='LA', count=52481),
 Row(State='OK', count=51297),
 Row(State='NJ', count=49942),
 Row(State='MD', count=43328),
 Row(State='UT', count=41385),
 Row(State='CO', count=40124),
 Row(State='AL', count=36369),
 Row(State='MA', count=33014),
 Row(State='IN', count=30040),
 Row(State='MO', count=29012),
 Row(State='CT', count=22803),
 Row(State='NE', count=22505),
 Row(State='KY', count=19122),
 Row(State='WI', count=17580),
 Row(State='RI', count=10483),
 R

In [17]:
if enabled == True:
    rdd = sc.parallelize(state_freq)
    pd_states = rdd.toDF().toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))
    b = sns.barplot(pd_states['State'],pd_states['count'], color='blue')
    b.axes.set_title("Severity distribution for each state",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("State",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"severity_dist_state")

In [18]:
state_severity_freq = df.groupBy('State','Severity').count().orderBy('count',ascending=False).collect()
state_severity_freq

[Row(State='CA', Severity=2, count=445381),
 Row(State='TX', Severity=2, count=215027),
 Row(State='CA', Severity=3, count=211189),
 Row(State='FL', Severity=2, count=144622),
 Row(State='NC', Severity=2, count=121261),
 Row(State='SC', Severity=2, count=114339),
 Row(State='NY', Severity=2, count=82388),
 Row(State='TX', Severity=3, count=79467),
 Row(State='FL', Severity=3, count=71631),
 Row(State='PA', Severity=2, count=67778),
 Row(State='OR', Severity=2, count=61538),
 Row(State='IL', Severity=2, count=58901),
 Row(State='MI', Severity=2, count=52583),
 Row(State='NY', Severity=3, count=49968),
 Row(State='AZ', Severity=2, count=47248),
 Row(State='OK', Severity=2, count=46194),
 Row(State='VA', Severity=2, count=42556),
 Row(State='GA', Severity=3, count=42393),
 Row(State='LA', Severity=2, count=41674),
 Row(State='TN', Severity=2, count=38526),
 Row(State='WA', Severity=2, count=37528),
 Row(State='MN', Severity=2, count=37017),
 Row(State='GA', Severity=2, count=34605),
 Row(

In [19]:
if enabled == True:
    rdd = sc.parallelize(state_severity_freq)
    pd_state_severity = rdd.toDF().toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))
    b = sns.barplot(x="State", y="count", hue="Severity", data=pd_state_severity)
    #b = sns.barplot(pd_state_severity['State', 'Severity'],pd_state_severity['count'])
    b.axes.set_title("Severity distribution for each state",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("State",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"severity_dist_class_state")

### Weather conditions

In [20]:
weather_freq = df.groupBy('Weather_Condition').count().orderBy('count',ascending=False).collect()
weather_freq

[Row(Weather_Condition='Clear', count=808171),
 Row(Weather_Condition='Mostly Cloudy', count=412528),
 Row(Weather_Condition='Overcast', count=382480),
 Row(Weather_Condition='Fair', count=335289),
 Row(Weather_Condition='Partly Cloudy', count=295439),
 Row(Weather_Condition='Scattered Clouds', count=204662),
 Row(Weather_Condition='Light Rain', count=141073),
 Row(Weather_Condition='Cloudy', count=115496),
 Row(Weather_Condition=None, count=65932),
 Row(Weather_Condition='Light Snow', count=42123),
 Row(Weather_Condition='Haze', count=34315),
 Row(Weather_Condition='Rain', count=32826),
 Row(Weather_Condition='Fog', count=22138),
 Row(Weather_Condition='Heavy Rain', count=12064),
 Row(Weather_Condition='Light Drizzle', count=10277),
 Row(Weather_Condition='Light Thunderstorms and Rain', count=4928),
 Row(Weather_Condition='Snow', count=4796),
 Row(Weather_Condition='Thunderstorm', count=4438),
 Row(Weather_Condition='Fair / Windy', count=3759),
 Row(Weather_Condition='Smoke', count=36

In [21]:

n = int(df.count()*0.001) # Limit the plot to ignore conditions below an limit

rdd = sc.parallelize(weather_freq)
rdd_filtered = rdd.filter(lambda x: x['count'] > n)
print(rdd_filtered.collect())

if enabled == True:
    fig, ax=plt.subplots(figsize=(16,25))
    pd_weather = rdd_filtered.toDF().toPandas()

    b = sns.barplot(pd_weather['count'][:],pd_weather['Weather_Condition'][:], color="blue")

    b.axes.set_title("Weather Condition for accidents above 1% of total set",fontsize=20)
    b.set_xlabel("Number of Accidents",fontsize=15)
    b.set_ylabel("Weather_Condition",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"weather_cond_dist")
    logger.write2file("Weather condition distribution", str(pd_weather))

[Row(Weather_Condition='Clear', count=808171), Row(Weather_Condition='Mostly Cloudy', count=412528), Row(Weather_Condition='Overcast', count=382480), Row(Weather_Condition='Fair', count=335289), Row(Weather_Condition='Partly Cloudy', count=295439), Row(Weather_Condition='Scattered Clouds', count=204662), Row(Weather_Condition='Light Rain', count=141073), Row(Weather_Condition='Cloudy', count=115496), Row(Weather_Condition=None, count=65932), Row(Weather_Condition='Light Snow', count=42123), Row(Weather_Condition='Haze', count=34315), Row(Weather_Condition='Rain', count=32826), Row(Weather_Condition='Fog', count=22138), Row(Weather_Condition='Heavy Rain', count=12064), Row(Weather_Condition='Light Drizzle', count=10277), Row(Weather_Condition='Light Thunderstorms and Rain', count=4928), Row(Weather_Condition='Snow', count=4796), Row(Weather_Condition='Thunderstorm', count=4438), Row(Weather_Condition='Fair / Windy', count=3759), Row(Weather_Condition='Smoke', count=3602)]


### Time when accidents occured

In [22]:
df_time = df.selectExpr("hour(to_timestamp(from_utc_timestamp(Start_Time, Timezone), 'yyyy-MM-dd HH:mm:ss')) as Start_Time")
time_freq = df_time.groupBy('Start_Time').count().orderBy('count',ascending=False).collect()
time_freq

[Row(Start_Time=3, count=251893),
 Row(Start_Time=2, count=227588),
 Row(Start_Time=4, count=210291),
 Row(Start_Time=11, count=185877),
 Row(Start_Time=12, count=185042),
 Row(Start_Time=10, count=181391),
 Row(Start_Time=1, count=168954),
 Row(Start_Time=9, count=164708),
 Row(Start_Time=5, count=162704),
 Row(Start_Time=13, count=155228),
 Row(Start_Time=6, count=154872),
 Row(Start_Time=7, count=154414),
 Row(Start_Time=8, count=153547),
 Row(Start_Time=0, count=116025),
 Row(Start_Time=14, count=105595),
 Row(Start_Time=15, count=76873),
 Row(Start_Time=23, count=73396),
 Row(Start_Time=16, count=57092),
 Row(Start_Time=22, count=43593),
 Row(Start_Time=17, count=42607),
 Row(Start_Time=18, count=29586),
 Row(Start_Time=21, count=27565),
 Row(Start_Time=20, count=21421),
 Row(Start_Time=19, count=20910),
 Row(Start_Time=None, count=3163)]

In [23]:
if enabled == True:
    dd = sc.parallelize(time_freq)
    pd_time = rdd.toDF().toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))

    b = sns.barplot(pd_time['Start_Time'],pd_time['count'], color='blue')
    b.axes.set_title("Daytime for accidents",fontsize=20)
    b.set_xlabel("Time (Hours)",fontsize=15)
    b.set_ylabel("Number of Accidents",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"accident_hours")

### Development of accidents on Month basis

In [24]:
df_time = df.selectExpr("month(to_timestamp(from_utc_timestamp(Start_Time, Timezone), 'yyyy-MM-dd HH:mm:ss')) as Start_Time")
time_freq = df_time.groupBy('Start_Time').count().orderBy('count',ascending=False).collect()
time_freq

[Row(Start_Time=10, count=324135),
 Row(Start_Time=12, count=299258),
 Row(Start_Time=11, count=298801),
 Row(Start_Time=9, count=292194),
 Row(Start_Time=8, count=288566),
 Row(Start_Time=7, count=222828),
 Row(Start_Time=4, count=211588),
 Row(Start_Time=6, count=209902),
 Row(Start_Time=1, count=209776),
 Row(Start_Time=5, count=208812),
 Row(Start_Time=3, count=206805),
 Row(Start_Time=2, count=198507),
 Row(Start_Time=None, count=3163)]

In [25]:
if enabled == True:
    rdd = sc.parallelize(time_freq)
    pd_time = rdd.toDF().toPandas()

    # Plot data
    fig,ax = plt.subplots(figsize=(16,10))

    b = sns.barplot(pd_time['Start_Time'],pd_time['count'], color='blue')
    b.axes.set_title("Montly distribution for accidents",fontsize=20)
    b.set_xlabel("Time (Month)",fontsize=15)
    b.set_ylabel("Number of Accidents",fontsize=15)
    b.tick_params(labelsize=10)

    logger.saveImage(b,"accudents_months")

### Map distribution

In [26]:
if enabled == True:
    b = sns.jointplot(x=df.select(collect_list('Start_Lat')).first()[0],y=df.select(collect_list('Start_Lng')).first()[0],height=10)

    b.set_axis_labels('Start_Lat','Start_Lng')
    b.fig.suptitle("Map distribution of accidents")
    logger.saveImage(b,"accident_map_dist")

In [27]:
logger.write2file("New Analysis created!", \
               "Model finished: Yes" + \
               "\nFolder name: " + logger.timeSignature + \
               "\nState: "+ "Not defined" + \
               "\nLogs directory: " + logger.logs_dir +  \
               "\nFile: " + file + \
               "\nNote: " + model_Note, \
               logs_dir=model_indexing_dir \
              )